In [1]:
from sympy import *
init_printing()

In [2]:
def skew(l):
    l1, l2, l3 = l
    return Matrix([
        [0, -l3, l2],
        [l3, 0, -l1],
        [-l2, l1, 0]
    ])

In [3]:
# define state variables
x, y, z, phi, theta, psi, u, v, w, p, q, r = symbols('x y z phi theta psi u v w p q r', real=True)
s = Matrix([x, y, z, phi, theta, psi, u, v, w, p, q, r])

# position and orientation
eta = Matrix([x, y, z, phi, theta, psi])
nu = Matrix([u, v, w, p, q, r])

# centre of gravity
xg, yg, zg = symbols('xg yg zg', real=True)
rg = Matrix([xg, yg, zg])

# centre of bouyancy
xb, yb, zb = symbols('xb yb zb', real=True)
rb = Matrix([xb, yb, zb])

# center of pressure
xcp, ycp, zcp = symbols('xcp ycp zcp', real=True)
rcp = Matrix([xcp, ycp, zcp])

# mass matrix
m = symbols('m', real=True, positive=True)
Ixx, Iyy, Izz = symbols('Ixx Iyy Izz')
I0 = diag(Ixx, Iyy, Izz)
M = BlockMatrix([
    [m*eye(3), -m*skew(rg)],
    [m*skew(rg), I0]
])
M = Matrix(M)
# M = simplify(M)

# Coriolis and centripetal matrix
nu1 = Matrix([u, v, w])
nu2 = Matrix([p, q, r])
crb = BlockMatrix([
    [zeros(3), -m*skew(nu1)-m*skew(nu2)*skew(rg)],
    [-m*skew(nu1)+m*skew(rg)*skew(nu2), -skew(I0*nu2)]
])
crb = Matrix(crb)
# crb = simplify(crb)

# damping matrix
Xuu, Yvv, Zww, Kpp, Mqq, Nrr = symbols(
    'Xuu Yvv Zww Kpp Mqq Nrr', real=True
)
D = Matrix([
    [Xuu*abs(u), 0, 0, 0, 0, 0],
    [0, Yvv*abs(v), 0, 0, 0, 0],
    [0, 0, Zww*abs(w), 0, 0, 0],
    [0, -zcp*Yvv*abs(v), ycp*Zww*abs(w), Kpp*abs(p), 0, 0],
    [zcp*Xuu*abs(u), 0, -xcp*Zww*abs(w), 0, Mqq*abs(q), 0],
    [-ycp*Xuu*abs(u), xcp*Yvv*abs(v), 0, 0, 0, Nrr*abs(r)]
])

# rotational transform between body and NED quaternions
# Tq = Rational(1,2)*Matrix([
#     [-eps1, -eps2, -eps3],
#     [eta0, -eps3, eps2],
#     [eps3, eta0, -eps1],
#     [-eps2, eps1, eta0]
# ])
T_euler = Matrix([
    [1, sin(phi)*tan(theta), cos(phi)*tan(theta)],
    [0, cos(phi), -sin(phi)],
    [0, sin(phi)/cos(theta), cos(phi)/cos(theta)]
])
 

# Rq = Matrix([
#     [1-2*(eps2**2+eps3**2), 2*(eps1*eps2-eps3*eta0), 2*(eps1*eps3+eps2*eta0)],
#     [2*(eps1*eps2+eps3*eta0), 1-2*(eps1**2+eps3**2), 2*(eps2*eps3-eps1*eta0)],
#     [2*(eps1*eps3-eps2*eta0), 2*(eps2*eps3+eps1*eta0), 1-2*(eps1**2+eps2**2)]
# ])
R_euler = Matrix([
    [cos(psi)*cos(theta), -sin(psi)*cos(phi)+cos(psi)*sin(theta)*sin(phi), sin(psi)*sin(phi)+cos(psi)*cos(phi)*sin(theta)],
    [sin(psi)*cos(theta), cos(psi)*cos(phi)+sin(phi)*sin(theta)*sin(psi), -cos(psi)*sin(phi)+sin(theta)*sin(psi)*cos(phi)],
    [-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi)]
])


# Jeta = BlockMatrix([
#     [Rq, zeros(3)],
#     [zeros(4,3), Tq]
# ])
Jeta = BlockMatrix([
    [R_euler, zeros(3)],
    [zeros(3), T_euler]
])


Jeta = Matrix(Jeta)
# Jeta = simplify(Jeta)

# bouyancy in quaternions
# W, B = symbols('W B', real=True)
# fg = Matrix([0, 0, W])
# fb = Matrix([0, 0, -B])
# Rqinv = Rq.inv()
# geta = Matrix([
#     Rqinv*(fg+fb),
#     skew(rg)*Rqinv*fg + skew(rb)*Rqinv*fb
# ])
W, B = symbols('W B', real=True)
fg = Matrix([0, 0, W])
fb = Matrix([0, 0, -B])
R_euler_inv = R_euler.inv()
geta = Matrix([
    R_euler_inv*(fg+fb),
    skew(rg)*R_euler_inv*fg + skew(rb)*R_euler_inv*fb
])
# geta = simplify(geta)

In [4]:
Jeta

⎡cos(ψ)⋅cos(θ)  sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)  sin(φ)⋅sin(ψ) + sin(θ)⋅c
⎢                                                                             
⎢sin(ψ)⋅cos(θ)  sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(ψ) + sin(ψ)⋅
⎢                                                                             
⎢   -sin(θ)                sin(φ)⋅cos(θ)                          cos(φ)⋅cos(θ
⎢                                                                             
⎢      0                         0                                      0     
⎢                                                                             
⎢      0                         0                                      0     
⎢                                                                             
⎢                                                                             
⎢      0                         0                                      0     
⎣                                                   

In [5]:
# thrust model
Kt0, Kt1 = symbols('Kt0 Kt1', real=True)
Kt = Matrix([Kt0, Kt1])
Qt0, Qt1 = symbols('Qt0 Qt1', real=True)
Qt = Matrix([Qt0, Qt1])

# control inputs
rpm0, rpm1 = symbols('rpm0 rpm1', real=True)
rpm = Matrix([rpm0, rpm1])
de, dr = symbols('de dr', real=True)
control_vector = Matrix([rpm0, rpm1, de, dr])

# control force vector
Ft = Kt.dot(rpm)
Mt = Qt.dot(rpm)

# coefficient for each element in cost function
tauc = Matrix([
    Ft*cos(de)*cos(dr),
    -Ft*sin(dr),
    Ft*sin(de)*cos(dr),
    Mt*cos(de)*cos(dr),
    -Mt*sin(dr),
    Mt*sin(de)*cos(dr)
])

In [6]:
etadot = Jeta*nu
nudot = M.inv()*(tauc - (crb + D)*nu - geta)

In [7]:
sdot = Matrix([
   etadot,
   nudot 
])

In [8]:
print(cse(sdot))

([(x0, cos(psi)), (x1, cos(theta)), (x2, u*x1), (x3, sin(phi)), (x4, sin(psi)), (x5, x3*x4), (x6, sin(theta)), (x7, cos(phi)), (x8, x0*x7), (x9, x4*x7), (x10, x0*x3), (x11, x1*x3), (x12, x1*x7), (x13, tan(theta)), (x14, q*x3), (x15, r*x7), (x16, 1/x1), (x17, Yvv*Abs(v)), (x18, m*xg), (x19, p*x18), (x20, m*zg), (x21, r*x20), (x22, m*u), (x23, m*yg), (x24, r*x23), (x25, sin(dr)), (x26, Kt0*rpm0), (x27, Kt1*rpm1), (x28, m*w), (x29, -x28), (x30, p*x23), (x31, -B + W), (x32, x3**2), (x33, x6**2), (x34, x1**2), (x35, x7**2), (x36, 1/(x32*x33 + x32*x34 + x33*x35 + x34*x35)), (x37, x11*x36), (x38, -p*(x29 - x30) - q*(x19 + x21) - r*(x22 - x24) - v*x17 + x25*(-x26 - x27) - x31*x37), (x39, m**2), (x40, xg**4), (x41, x39*x40), (x42, yg**2), (x43, xg**2), (x44, x39*x43), (x45, x42*x44), (x46, zg**2), (x47, x44*x46), (x48, Iyy*Izz), (x49, Ixx*x48), (x50, Ixx*Izz), (x51, m*x50), (x52, m*x48), (x53, -x42*x52 - x43*x51 + x49), (x54, Ixx*Iyy), (x55, m*x54), (x56, -x43*x55 - x46*x52), (x57, Ixx*x41 + Iy